In [67]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay
from prophet import Prophet

In [68]:
# Step 1: Load and Preprocess Data from Excel
def load_and_preprocess_data(file_path):
    # Load Excel file
    df = pd.read_excel(file_path)
    # Print column names for debugging
    print("Original column names:", df.columns.tolist())
    # Strip whitespace from column names
    df.columns = df.columns.str.strip()
    # Rename columns to match expected format
    column_map = {
        'Date': 'Date',
        'Open': 'Open',
        'High': 'High',
        'Low': 'Low',
        'Close*': 'Close',  # Adjust based on your actual column name
        'Adj Close**': 'Adj Close',  # Adjust based on your actual column name
        'Volume': 'Volume'
    }
    # Update column names
    df = df.rename(columns={k: v for k, v in column_map.items() if k in df.columns})
    print("Renamed column names:", df.columns.tolist())
    # Convert 'Date' to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    return df

In [69]:
# Step 2: Calculate Financial Indicators
def calculate_indicators(df):
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['EMA_5'] = df['Close'].ewm(span=5, adjust=False).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=5).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=5).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['BB_Middle'] = df['Close'].rolling(window=5).mean()
    df['BB_Std'] = df['Close'].rolling(window=5).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
    return df

In [70]:
def detect_anomalies(df):
    features = ['Close', 'SMA_5', 'EMA_5', 'RSI', 'BB_Upper', 'BB_Lower', 'Volume']
    # Filter rows with no NaN values in features
    df_features = df[features].dropna()
    # Keep track of valid indices
    valid_indices = df_features.index
    # Scale features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_features)
    # Apply Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    anomalies = iso_forest.fit_predict(scaled_features)
    # Map predictions: 1 for anomaly, 0 for normal
    anomalies = pd.Series(anomalies, index=valid_indices).map({1: 0, -1: 1})
    # Initialize 'Anomaly' column with NaN
    df['Anomaly'] = np.nan
    # Assign predictions only to valid rows
    df.loc[valid_indices, 'Anomaly'] = anomalies
    return df

In [71]:
# Step 4: Time-Series Forecasting with Prophet
def forecast_with_prophet(df):
    prophet_df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
    model = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
    model.add_seasonality(name='custom', period=5, fourier_order=3)
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=5, freq=BDay())
    forecast = model.predict(future)
    df['Forecast'] = forecast['yhat'][:len(df)]
    df['Deviation'] = np.abs(df['Close'] - df['Forecast'])
    threshold = df['Deviation'].mean() + 2 * df['Deviation'].std()
    df['Forecast_Anomaly'] = (df['Deviation'] > threshold).astype(int)
    return df, forecast

In [72]:
# Step 5: Visualize Results
def visualize_results(df, forecast):
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
    plt.plot(df['Date'], df['Forecast'], label='Prophet Forecast', color='green', linestyle='--')
    anomalies = df[df['Anomaly'] == 1]
    plt.scatter(anomalies['Date'], anomalies['Close'], color='red', label='Anomalies (Isolation Forest)', marker='o')
    forecast_anomalies = df[df['Forecast_Anomaly'] == 1]
    plt.scatter(forecast_anomalies['Date'], forecast_anomalies['Close'], color='orange',
                label='Anomalies (Forecast Deviation)', marker='x')
    plt.plot(df['Date'], df['BB_Upper'], label='Bollinger Upper', color='gray', linestyle='--')
    plt.plot(df['Date'], df['BB_Lower'], label='Bollinger Lower', color='gray', linestyle='--')
    plt.title('Stock Price Anomaly Detection')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.savefig('anomaly_detection.png')
    plt.close()

In [73]:
if __name__ == "__main__":
    df = load_and_preprocess_data('/yahoo_data.xlsx')
    df = calculate_indicators(df)
    df = detect_anomalies(df)
    df, forecast = forecast_with_prophet(df)
    visualize_results(df, forecast)

    # Report
    print("Anomaly Detection Report")
    print("======================")
    print("Dataset Overview:")
    print(df.head())
    print("\nDetected Anomalies (Isolation Forest):")
    print(df[df['Anomaly'] == 1][['Date', 'Close', 'Volume', 'RSI']])
    print("\nDetected Anomalies (Forecast Deviation):")
    print(df[df['Forecast_Anomaly'] == 1][['Date', 'Close', 'Forecast', 'Deviation']])
    print("\nVisualization saved as 'anomaly_detection.png'")

Original column names: ['Date', 'Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume']
Renamed column names: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


DEBUG:cmdstanpy:input tempfile: /tmp/tmp0siee7_b/hkgz4bpn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0siee7_b/hdp1cpia.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=71216', 'data', 'file=/tmp/tmp0siee7_b/hkgz4bpn.json', 'init=/tmp/tmp0siee7_b/hdp1cpia.json', 'output', 'file=/tmp/tmp0siee7_b/prophet_modelo28jxpyi/prophet_model-20250610064355.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:43:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:43:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Anomaly Detection Report
Dataset Overview:
           Date      Open      High       Low     Close  Adj Close     Volume  \
1257 2018-05-01  24117.29  24117.29  23808.19  24099.05   24099.05  380070000   
1256 2018-05-02  24097.63  24185.52  23886.30  23924.98   23924.98  385350000   
1255 2018-05-03  23836.23  23996.15  23531.31  23930.15   23930.15  389240000   
1254 2018-05-04  23865.22  24333.35  23778.87  24262.51   24262.51  329480000   
1253 2018-05-07  24317.66  24479.45  24263.42  24357.32   24357.32  307670000   

          SMA_5         EMA_5        RSI  BB_Middle      BB_Std      BB_Upper  \
1257        NaN  24099.050000        NaN        NaN         NaN           NaN   
1256        NaN  24041.026667        NaN        NaN         NaN           NaN   
1255        NaN  24004.067778        NaN        NaN         NaN           NaN   
1254        NaN  24090.215185        NaN        NaN         NaN           NaN   
1253  24114.802  24179.250123  71.294998  24114.802  194.299712  